# ABS Monthly Labour Force 6202

## Python set-up

In [1]:
# Python
from functools import cache


# Analytic imports
import pandas as pd
import numpy as np
import sdmxabs as sa
from mgplot import (
    clear_chart_dir,
    seastrend_plot_finalise,
    multi_start,
    series_growth_plot_finalise,
    growth_plot_finalise,
    summary_plot_finalise,
    line_plot_finalise,
    set_chart_dir,
)
from sdmxabs import MatchType as Mt

# display charts in this notebook
SHOW_CHARTS = False

# save charts in this notebook
CHART_DIR = "./CHARTS/Monthly-Labour-Force/"
set_chart_dir(CHART_DIR)
clear_chart_dir()

## Exploratory

### Data flows

In [2]:
def flows_of_interest() -> pd.DataFrame:
    """
    Returns a DataFrame of data flows of interest for the Monthly Labour Force dataset.
    """
    flows = sa.frame((sa.data_flows()))
    interesting = flows[flows.name.str.lower().str.contains("labour force") &
                        ~flows.name.str.lower().str.contains("census") &
                        ~flows.index.str.lower().str.contains("census")]
    return interesting

INTERESTING_FLOWS = flows_of_interest()
INTERESTING_FLOWS

,agencyID,version,isFinal,name
LF,ABS,1.0.0,true,Labour Force
LF_AGES,ABS,1.0.0,true,Labour Force: Age Groups
LF_EDU,ABS,1.0.0,true,Labour Force Educational Attendance
LF_HOURS,ABS,1.0.0,true,Labour Force: Hours worked by sector
LF_UNDER,ABS,1.0.1,true,Labour force - underemployment and underutilis...


### Set FLOW_ID and RFOOTER

In [3]:
FLOW_ID = "LF"
RFOOTER = f"ABS {INTERESTING_FLOWS.at[FLOW_ID, 'name']}"

### Dimensions

In [4]:
sa.frame(sa.data_dimensions(FLOW_ID))

,position,id,version,agencyID,package,class
MEASURE,1,CL_LF_MEASURE,1.0.0,ABS,codelist,Codelist
SEX,2,CL_SEX,1.0.0,ABS,codelist,Codelist
AGE,3,CL_LF_AGE,1.0.0,ABS,codelist,Codelist
TSEST,4,CL_TSEST,1.0.0,ABS,codelist,Codelist
REGION,5,CL_STATE,1.0.0,ABS,codelist,Codelist
FREQ,6,CL_FREQ,1.0.0,ABS,codelist,Codelist
UNIT_MEASURE,NaN,CL_UNIT_MEASURE,1.0.0,ABS,codelist,Codelist
UNIT_MULT,NaN,CL_UNIT_MULT,1.0.0,ABS,codelist,Codelist
OBS_STATUS,NaN,CL_OBS_STATUS,1.0.0,ABS,codelist,Codelist
OBS_COMMENT,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
sa.frame(sa.data_dimensions("LF_HOURS"))

,position,id,version,agencyID,package,class
MEASURE,1,CL_LF_HOURS_MEASURE,1.0.0,ABS,codelist,Codelist
SEX,2,CL_SEX,1.0.0,ABS,codelist,Codelist
AGE,3,CL_LF_AGE,1.0.0,ABS,codelist,Codelist
HOURS,4,CL_LF_HOURS,1.0.0,ABS,codelist,Codelist
TSEST,5,CL_TSEST,1.0.0,ABS,codelist,Codelist
REGION,6,CL_STATE,1.0.0,ABS,codelist,Codelist
FREQ,7,CL_FREQ,1.0.0,ABS,codelist,Codelist
UNIT_MEASURE,NaN,CL_UNIT_MEASURE,1.0.0,ABS,codelist,Codelist
UNIT_MULT,NaN,CL_UNIT_MULT,1.0.0,ABS,codelist,Codelist
OBS_STATUS,NaN,CL_OBS_STATUS,1.0.0,ABS,codelist,Codelist


### Code lists

In [6]:
sa.frame(sa.code_list_for_dim(FLOW_ID, "MEASURE"))

,name
M1,Employed - full-time
M2,Employed - part-time
M3,Employed persons
M4,Unemployed - looking for full-time work
M5,Unemployed - looking for part-time work
M6,Unemployed persons
M7,Labour Force - Full-time
M8,Labour Force - Part-time
M9,Labour Force
M10,Not in the Labour Force


In [7]:
sa.frame(sa.code_list_for_dim("LF_HOURS", "MEASURE"))

,name
M1,Employed - full-time
M29,Underemployed total (expanded analytical series)
M28,Underemployed part-time (expanded analytical s...
M27,Underemployed full-time (expanded analytical s...
M26,Underemployed part-time (prefer more hours)
M25,Underemployed full-time (worked part-time for ...
M24,Underutilisation rate
M23,Underemployment rate (proportion of labour force)
M22,Underemployment ratio (proportion of employed)
M21,Underemployed total


## Utility functions

In [8]:
#@ cache
def fetch_headline_data() -> tuple[pd.DataFrame, pd.DataFrame]:
    """Fetch the Seasonally Adjusted and Trend headline data for the Monthly Labour Force."""

    # --- fetch the headline data for the Monthly Labour Force
    endswith = "rate", "Force", "persons", "ratio", "population"
    wanted = [f"^{x}$" for x in sa.frame(sa.code_list_for_dim(FLOW_ID, "MEASURE")).name
            if any(x.endswith(e) for e in endswith)]    
    selection = [
        ("Monthly", "FREQ", Mt.EXACT),
        ("Persons", "SEX", Mt.EXACT),
        ("Australia", "REGION", Mt.EXACT),
        ("Total (ages)", "AGE", Mt.EXACT),
        ("|".join(wanted), "MEASURE", Mt.REGEX),
    ]
    d, m = sa.fetch_selection(FLOW_ID, selection)

    # --- add in hours worked data
    selection = [
        ("Monthly", "FREQ", Mt.EXACT),
        ("Employed Persons - Monthly hours worked in all jobs", "MEASURE", Mt.EXACT),
        ("Australia", "REGION", Mt.EXACT),
        ("Persons", "SEX", Mt.EXACT),
    ]
    d1, m1 = sa.fetch_selection("LF_HOURS", selection)

    # --- add in underemployment data


    return d.join(d1, how="outer"), m.T.join(m1.T, how="outer").T

## Headline Seasonal/Trend Charts

In [9]:
def plot_headline_st():

    data, meta = fetch_headline_data()
    unique = meta["MEASURE"].unique()
    a_bit_over_three_years_ago = -38  # to look at recent data.

    for u in unique:

        # --- collect the data
        use_seasonal = True
        orig = seas = tr = ""
        try:
            seas = meta[(meta["MEASURE"] == u) & (meta["TSEST"] == "Seasonally Adjusted")].index[0]
            tr = meta[(meta["MEASURE"] == u) & (meta["TSEST"] == "Trend")].index[0]
        except IndexError:
            orig = meta[(meta["MEASURE"] == u) & (meta["TSEST"] == "Original")].index[0]
            use_seasonal = False

        # --- if seasonal/trend data that is what we will chart ...
        if use_seasonal:
            # --- collect terms
            plot_data = data[[seas, tr]].dropna(axis=1, how="all")
            plot_meta = meta.loc[[seas, tr]]
            units = sa.measure_names(plot_meta)
            recal_data, labels = sa.recalibrate(plot_data, units, as_a_whole=True)
            recal_data.columns = ["Seasonally Adjusted", "Trend"]

            # --- plot the seasonally adjusted and trend data
            multi_start(
                recal_data,
                function=seastrend_plot_finalise,
                starts=[0, a_bit_over_three_years_ago],
                title=u,
                ylabel=labels.iloc[0],
                rfooter=RFOOTER,
                lfooter="Australia. ",
                show=SHOW_CHARTS,
            )
            continue

        # --- original data that has not been seasonally adjusted
        units = sa.measure_names(meta.loc[[orig]])
        p, label = sa.recalibrate_series(data[orig], units[orig])
        multi_start(
            p,
            function=line_plot_finalise,
            starts=[0, a_bit_over_three_years_ago],
            title=u,
            ylabel=label,
            rfooter=RFOOTER,
            lfooter="Australia. Original series. ",
            annotate=True,
            show=SHOW_CHARTS,
        )
    

plot_headline_st()

## Headline Growth

In [10]:
def plot_headline_growth() -> None:
    """Plot the percentage growth and numerical growth for the headline data."""

    # --- fetch the data
    data, meta = fetch_headline_data()
    units = sa.measure_names(meta)
    sadjust = meta[
        (meta["TSEST"] == "Seasonally Adjusted")
        & (meta["UNIT_MEASURE"].isin(["Number", "Hours"]))
    ].index
    seas_data = data[sadjust]

    # --- plot the percentage growth and numerical growth for each column
    for column in seas_data.columns:

        # --- preliminaries
        a_bit_over_a_year = -16  # months - to look over recent data.
        meta_row = meta.loc[column]
        lfooter = f"{meta_row['REGION']}. {meta_row['TSEST']}"
        name = "Monthly Hours Worked" if "hours" in meta_row["MEASURE"].lower() else meta_row["MEASURE"]
        title_stem = f" growth: {name} ({meta_row.at['REGION']})"

        # --- calculate and plot percentage growth
        series_growth_plot_finalise(
            seas_data[column],
            plot_from=a_bit_over_a_year,
            title=f"Percentage{title_stem}",
            rfooter=RFOOTER,
            lfooter=lfooter,
            bar_rounding=2,
            show=SHOW_CHARTS,
        )

        # --- calculate and plot numerical growth
        monthly = seas_data[column].diff(1).dropna()
        annual = seas_data[column].diff(12).dropna()
        plot_data = pd.DataFrame([annual, monthly], index=["Annual", "Monthly"]).T.sort_index()
        ounits = pd.Series(
            [units[column]] * 2, 
            index=plot_data.columns
        )
        plot_data, ounits = sa.recalibrate(plot_data, ounits, as_a_whole=True)
        growth_plot_finalise(
            plot_data,
            plot_from=a_bit_over_a_year,
            title=f"Numerical{title_stem}",
            ylabel=ounits.iloc[0],
            rfooter=RFOOTER,
            lfooter=lfooter,
            bar_rounding=0,
            show=SHOW_CHARTS,
        )


plot_headline_growth()

## Summary

In [11]:
def plot_summary(start=pd.Period("2000-01"), freq='M') -> None:

    data, meta = fetch_headline_data()
    sa_rows = meta[meta["TSEST"] == "Seasonally Adjusted"].index
    sa_data = data[sa_rows]
    final = {}
    for column in sa_data.columns:
        meta_row = meta.loc[column]
        series = sa_data[column].dropna()
        if meta_row.at["UNIT_MEASURE"] == "Percent":
            final[meta_row["MEASURE"]] = series
            continue
        if meta_row.at["UNIT_MEASURE"] in ("Number", "Hours"):
            name = meta_row["MEASURE"] if "hours" not in meta_row["MEASURE"] else "Monthly hours worked."
            final[f"{name} 12m growth"] = series.pct_change(12) * 100
            final[f"{name} 1m growth"] = series.pct_change(1) * 100
    summary_frame = pd.DataFrame(final).sort_index()
    summary_plot_finalise(
        summary_frame.loc[summary_frame.index >= start],
        title=f"Monthly Labour Force Summary For {summary_frame.index[-1].strftime('%B %Y')}",
        rfooter=RFOOTER,
        lfooter="Australia. Seasonally Adjusted. Values in black text are percentages. ",
        show=SHOW_CHARTS,
    )


plot_summary()

## Sahm Rule

The Sahm Rule is designed to provide an indicator for when the economy has entered a recession, noting that GDP based indications of a recession are usually late to surface due to lags in data reporting. 

To calculate the Sahm Rule number, one compares the current three-month moving average to the lowest three-month moving average from the prior twelve months -- if that result is more than 0.50, then there is an extremely strong likelihood that we are in a recession.

_Note_: Arguable that the 50 basis point threshold is too low for Australia. Historically, 60 basis points looks like a better threshold

## Finished

In [12]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-07-15 21:02:01

Python implementation: CPython
Python version       : 3.13.5
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pandas : 2.3.1
mgplot : 0.2.6
sdmxabs: 0.1.9a4
numpy  : 2.3.1

Watermark: 2.5.0



In [13]:
print("Finished")

Finished
